# the cryptopals crypto challenges Set 1

## Convert hex to base64

The string:
49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d

Should produce:
SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t

So go ahead and make that happen. You'll need to use this code for the rest of the exercises.

### Cryptopals Rule!!
Always operate on raw bytes, never on encoded strings. Only use hex and base64 for pretty-printing.

In [1]:
import base64


hexstring = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
base64string = 'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

def hex_to_base46(hex_string: str) -> str:
    raw_bytes = bytes.fromhex(hex_string)
    return base64.b64encode(raw_bytes).decode()

res = hex_to_base46(hexstring)
print(res)
print(f'{res == base64string}')

SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t
True


## Fixed XOR

Write a function that takes two equal-length buffers and produces their XOR combination.

If your function works properly, then when you feed it the string:

1c0111001f010100061a024b53535009181c

... after hex decoding, and when XOR'd against:

686974207468652062756c6c277320657965

... should produce:

746865206b696420646f6e277420706c6179

In [2]:
buffer_1 = '1c0111001f010100061a024b53535009181c'
buffer_2 ='686974207468652062756c6c277320657965'
expected_result = '746865206b696420646f6e277420706c6179'

def xor_hex_strings(hex_string_1: str, hex_string_2: str) -> str:
    if len(hex_string_1) != len(hex_string_2):
        raise ValueError("Input hex strings must have the same length.")

    raw_bytes_1 = bytes.fromhex(hex_string_1)
    raw_bytes_2 = bytes.fromhex(hex_string_2)
    res = bytes(x ^ y for x ,y  in zip(raw_bytes_1, raw_bytes_2))
    return res.hex()

res = xor_hex_strings(buffer_1, buffer_2)
print(res)
print(expected_result == res)

746865206b696420646f6e277420706c6179
True


## Single-byte XOR cipher

The hex encoded string:

1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736

... has been XOR'd against a single character. Find the key, decrypt the message.

You can do this by hand. But don't: write code to do it for you.

How? Devise some method for "scoring" a piece of English plaintext. Character frequency is a good metric. Evaluate each output and choose the one with the best score.

In [3]:


hex_string = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'

english_freq = {
    'a': 8.167, 'b': 1.492, 'c': 2.782, 'd': 4.253, 'e': 12.702,
    'f': 2.228, 'g': 2.015, 'h': 6.094, 'i': 6.966, 'j': 0.153,
    'k': 0.772, 'l': 4.025, 'm': 2.406, 'n': 6.749, 'o': 7.507,
    'p': 1.929, 'q': 0.095, 'r': 5.987, 's': 6.327, 't': 9.056,
    'u': 2.758, 'v': 0.978, 'w': 2.360, 'x': 0.150, 'y': 1.974,
    'z': 0.074, ' ': 13.000  # Including space as it's common in English text
}

def score_text_on_character_frequency( text: str ) -> float:
    return sum( english_freq.get(char, 0) for char in text )

def single_byte_xor_decrypt( text_bytes: bytes, key: int ) -> bytes:
    return bytes( byte ^ key for byte in text_bytes )

def guess_xor_key(hex_cypher_text: str) -> tuple[str, int, float]:
    raw_byes = bytes.fromhex(hex_cypher_text)
    best_text = ''
    best_key = 0
    best_score = 0

    for i in range(256):
        try:
            decrypted = single_byte_xor_decrypt(raw_byes, i)
            text = decrypted.decode()
            score = score_text_on_character_frequency(text)
            if score > best_score:
                best_score = score
                best_text = text
                best_key = i
        except UnicodeDecodeError:
            continue
    return best_text, best_key, best_score


text, key, score = guess_xor_key(hex_string)
print(text)
print(key)
print(key.to_bytes().decode())

post = 'ETAOIN SHRDLU'.lower()

for c in post:
    print(f"character: {c}, score: {english_freq[c]}")

Cooking MC's like a pound of bacon
88
X
character: e, score: 12.702
character: t, score: 9.056
character: a, score: 8.167
character: o, score: 7.507
character: i, score: 6.966
character: n, score: 6.749
character:  , score: 13.0
character: s, score: 6.327
character: h, score: 6.094
character: r, score: 5.987
character: d, score: 4.253
character: l, score: 4.025
character: u, score: 2.758




## Detect single-character XOR

One of the 60-character strings in this 4.txt has been encrypted by single-character XOR.

Find it.

In [4]:
with open('4.txt') as file:
    lines = [line.rstrip() for line in file]

decoded = [guess_xor_key(line) for line in lines]
sorted_decoded = sorted(decoded, key=lambda x: x[2], reverse=True)
print(sorted_decoded[0][0])

Now that the party is jumping

